# 1. Tiền xử lý dữ liệu sơ bộ

In [1]:
import os
import json
import re
import numpy as np
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import difflib

# Product

In [2]:
products = pd.read_csv('ProductRaw.csv')

In [3]:
products.head()

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.7,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.8,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [4]:
print('Số lượng sản phẩm: ', products.shape[0])

Số lượng sản phẩm:  4404


In [5]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4404 non-null   int64  
 1   name         4404 non-null   object 
 2   description  4401 non-null   object 
 3   rating       4404 non-null   float64
 4   price        4404 non-null   int64  
 5   list_price   4404 non-null   int64  
 6   brand        4404 non-null   object 
 7   group        4404 non-null   object 
 8   url          4404 non-null   object 
 9   image        4404 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 344.2+ KB


In [6]:
from pandas_profiling import ProfileReport
profile = ProfileReport(products)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:01<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# loại bỏ DL trùng:
print('Trước khi drop, số sản phẩm là: ', products.shape[0])
products = products.drop_duplicates()
print('Sau khi drop, số sản phẩm là: ', products.shape[0])

Trước khi drop, số sản phẩm là:  4404
Sau khi drop, số sản phẩm là:  4373


In [8]:
# kiểm tra dữ liệu null:
products.isnull().sum()

item_id        0
name           0
description    3
rating         0
price          0
list_price     0
brand          0
group          0
url            0
image          0
dtype: int64

Dữ liệu bị null tại description (3 records), các cột khác không bị null.

In [9]:
# lưu dữ liệu
products.to_csv('Product_new.csv', index = False)

# Review

In [10]:
reviews = pd.read_csv('ReviewRaw.csv')

In [11]:
reviews.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [12]:
print('Số lượng đánh giá: ', reviews.shape[0])

Số lượng đánh giá:  364099


In [13]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [14]:
profile1 = ProfileReport(reviews)
profile1

Summarize dataset:   0%|          | 0/5 [00:05<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:11<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
# loại bỏ DL trùng:
print('Trước khi drop, records = ', reviews.shape[0])
reviews.drop_duplicates(inplace = True)
print('Sau khi drop, records = ', reviews.shape[0])

Trước khi drop, records =  364099
Sau khi drop, records =  361750


In [16]:
# kiểm tra dữ liệu null:
reviews.isnull().sum()

customer_id          0
product_id           0
name               428
full_name        34603
created_time    245525
rating               0
title               29
content         196064
dtype: int64

In [17]:
m = []
for c in ['name', 'full_name', 'created_time', 'content', 'title']:
  m.append({
      'feature': c,
      'Missing_Value': reviews[reviews[c].isnull()].shape[0],
      'Percentage': reviews[reviews[c].isnull()].shape[0]/reviews.shape[0]*100
  })
df = pd.DataFrame(m)
df

,feature,Missing_Value,Percentage
0,name,428,0.118314
1,full_name,34603,9.565446
2,created_time,245525,67.871458
3,content,196064,54.198756
4,title,29,0.008017


### Nhận xét:
- Dữ liệu null nhiều ở created_time, content. 
- Cột title có giá trị null nhưng số lượng ít.
- name và full_name (nếu có) khá giống nhau. full_name có dẽ liệu null tương đối (9.5%)

In [18]:
# bỏ feature full_name và created_time
reviews.drop(['full_name', 'created_time'], axis = 1, inplace = True)

In [19]:
reviews.shape

(361750, 6)

In [20]:
# lưu kết quả:
reviews.to_csv('Review_new.csv', index = False)

# Product-Review

In [21]:
products = pd.read_csv('Product_new.csv')
reviews = pd.read_csv('Review_new.csv')

In [22]:
reviews.shape

(361750, 6)

In [23]:
# sp không có trong review
reviews[~reviews.product_id.isin(products.item_id)]

,customer_id,product_id,name,rating,title,content
155686,5687133,20845056,Lê Phương Tiến,4,Hài lòng,Hài lòng về máy sau 1 tuần dùng. Điểm trừ giao...
155687,8344149,20845056,Luu Tri Dung,5,Tuyệt vời,"Giao hàng đúng thời gian dự kiến, có 2 ngày. M..."
155688,10671105,20845056,Nguyễn Thu Hằng,5,Cực kì hài lòng,"Sản phẩm được đóng gói cẩn thận, pin dùng được..."
155700,373425,20845056,Quân Nguyễn,2,Nên mua ở ngoài,Mình mua kindle Paperwhite 2019 được tầm cỡ 2 ...
155708,1279789,20845056,Quân Trần,1,"Ship chậm, máy ko active được",Tôi nhận đc sản phẩm ko active được. Khi liên ...
...,...,...,...,...,...,...
191096,10449689,12706787,Hai An,4,Hài lòng,có dùng cho dòng điện thoại khác ngoài iphone ...
191097,6079274,12706787,Nguyễn trung hiếu,4,"Hàng nghe mượt, tốt",Hàng chuẩn! Nhưng mình đang phân vân ko biết 1...
191098,7474878,12706787,Mon Pham,5,Quá phê !!!!,Mong lần sau đừng giao hàng chậm nữa tháng như...
191099,13144258,12706787,Đỗ Đức Việt,5,Thắc mắc,Hàng nhập khẩu với hàng VNA khác nhau gì vậy T...


In [24]:
# loại bỏ review có mã sp không tồn tại
reviews = reviews[reviews.product_id.isin(products.item_id)]

In [25]:
reviews.reset_index(drop = True, inplace = True)

In [26]:
# lưu kết quả:
reviews.to_csv('Review_new.csv', index_label = 'id')